# Evaluation Notebook

In [21]:
import os
import pickle
import pandas as pd

from utils import tools

In [22]:
data = 'pvod'

config = tools.load_config('config.yaml')
results_dir = config['eval']['results_path']
results_dir = os.path.join(results_dir, data)
result_files = os.listdir(results_dir)
# pickle files where training was performed on single dataset sequentially
sims = [f for f in result_files if (f.endswith('.pkl')) & ('cl' not in f) & ('fl' not in f)]
# pickle files where training was performed on concatenated datasets (centralized learning)
cl_sims = [f for f in result_files if 'cl' in f]
# pickle files for federated learning simulations
fl_sims = [f for f in result_files if 'fl' in f]
persistence_file = [f for f in result_files if 'persistence' in f][0]

In [23]:
def concatenate_results(results_dir: str,
                        results: list,
                        get_skill: bool = False,
                        pers: pd.DataFrame = pd.DataFrame(),
                        sort_skill: bool = False) -> pd.DataFrame:
    index_cols = ['Models', 'output_dim', 'freq', 't_0']
    indices = {}
    metrics = []
    # initialize the dict list
    for col in index_cols:
        indices[col] = []
    for file in results:  # Iterate through all result files
        # Load the data
        with open(os.path.join(results_dir, file), 'rb') as f:
            pkl = pickle.load(f)
        df = pkl['evaluation']
        df.reset_index(inplace=True)
        if get_skill:
            df= pd.merge(df, pers[['RMSE', 'key', 'output_dim', 'freq']],
               on=['key', 'output_dim', 'freq'],
               how='left',
               suffixes=('', '_p'))
            df['Skill'] = 1 - df.RMSE / df.RMSE_p
            df.drop('RMSE_p', axis=1, inplace=True)
        for col in index_cols:
            indices[col].append(df[col].iloc[0])
        df.drop(index_cols, axis=1, inplace=True)
        df.drop('key', axis=1, inplace=True)
        metric = df.mean(axis=0)
        metrics.append(metric)
    df = pd.DataFrame(metrics, columns=metric.index)
    df_index = pd.DataFrame(indices)
    df = pd.concat([df, df_index], axis=1)
    df.sort_values(['output_dim', 'freq', 'Models'], inplace=True)
    if sort_skill:
        df.sort_values(['Skill'], ascending=False, inplace=True)
    return df

def read_sim(results_dir, sim):
    try:
        with open(os.path.join(results_dir, sim), 'rb') as f:
            sim_results = pickle.load(f)
    except:
        sim_results = None
    return sim_results

In [24]:
pers = pd.read_csv(os.path.join(results_dir, persistence_file))

Local simulation analysis - 1 dataset : 1 model 

In [25]:
df_sep = concatenate_results(results_dir=results_dir, results=sims)
df_sep

,R^2,RMSE,MAE,Skill,retrain_interval,Models,output_dim,freq,t_0
1,0.879178,0.092808,0.051545,0.390635,NaN,bilstm,1,1h,None
2,0.877271,0.093480,0.051344,0.387123,NaN,lstm,1,1h,None
4,0.865613,0.096584,0.058340,0.332225,NaN,tft,1,1h,None
5,0.866132,0.097908,0.059619,0.357072,NaN,bilstm,48,1h,None
6,0.837694,0.107862,0.064739,0.293578,NaN,lstm,48,1h,None
0,0.819761,0.113709,0.069235,0.257288,NaN,tcn-gru,48,1h,None
3,0.852033,0.100744,0.057613,0.298598,NaN,tft,48,1h,None


Centralized simulation analysis - N datasets : 1 model

In [29]:
df_cl = concatenate_results(results_dir=results_dir,
                             get_skill=True,
                             pers=pers,
                             results=cl_sims)
df_cl

,R^2,RMSE,MAE,Skill,Models,output_dim,freq,t_0
0,0.725657,0.130368,0.072616,0.110185,tft,48,1h,None


Federated simulation analysis - N datasets : 1 model

In [13]:
df_fl = concatenate_results(results_dir=results_dir,
                            results=fl_sims)
df_fl

/Users/viktorwalter/Code/Forecasting/forecasting_framework/frcst/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 118 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/viktorwalter/Code/Forecasting/forecasting_framework/frcst/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


,R^2,RMSE,MAE,Skill,Models,output_dim,freq,t_0
4,0.835251,0.107536,0.058138,0.295496,tcn-gru,1,1h,None
3,0.850806,0.102455,0.058093,0.334009,bilstm,48,1h,None
2,0.831738,0.108809,0.061448,0.290704,lstm,48,1h,None
1,0.840405,0.106631,0.061371,0.308183,tcn-gru,48,1h,None
0,0.856006,0.101952,0.056313,0.315888,tft,48,1h,None


Individual simulation analysis

In [33]:
model = 'tft'
output_dim = 48
freq = '1h'

sim = f'd-{data}_m-{model}_out-{output_dim}_freq-{freq}.pkl'
cl_sim = f'cl_d-{data}_m-{model}_out-48_freq-{freq}.pkl'
fl_sim = f'fl_d-{data}_m-{model}_out-{output_dim}_freq-{freq}.pkl'

In [34]:
sim_results = read_sim(results_dir, sim)
cl_results = read_sim(results_dir, cl_sim)
fl_results = read_sim(results_dir, fl_sim)

/Users/viktorwalter/Code/Forecasting/forecasting_framework/frcst/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 118 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [35]:
sim_results['evaluation']

,R^2,RMSE,MAE,Skill,retrain_interval,output_dim,freq,key,t_0
Models,,,,,,,,,
tft,0.840924,0.092173,0.051299,0.262130,None,48,1h,station00.csv,None
tft,0.868403,0.111276,0.064320,0.293956,None,48,1h,station01.csv,None
tft,0.872736,0.092309,0.048902,0.350687,None,48,1h,station02.csv,None
tft,0.890199,0.127418,0.080417,0.468176,None,48,1h,station04.csv,None
tft,0.735219,0.107519,0.067345,-0.052346,None,48,1h,station06.csv,None
tft,0.870250,0.088710,0.045031,0.366826,None,48,1h,station07.csv,None
tft,0.886501,0.085805,0.045977,0.400761,None,48,1h,station08.csv,None


In [32]:
fl_results['evaluation']

,R^2,RMSE,MAE,Skill,output_dim,freq,key,t_0
Models,,,,,,,,
tcn-gru,0.833649,0.093456,0.051756,0.311809,48,1h,station00.csv,None
tcn-gru,0.891547,0.100071,0.058483,0.352876,48,1h,station01.csv,None
tcn-gru,0.900359,0.081169,0.047257,0.446486,48,1h,station02.csv,None
tcn-gru,0.761073,0.187728,0.113404,0.230867,48,1h,station04.csv,None
tcn-gru,0.750553,0.106124,0.061374,0.029743,48,1h,station06.csv,None
tcn-gru,0.873200,0.087860,0.048575,0.393831,48,1h,station07.csv,None
tcn-gru,0.872457,0.090008,0.048747,0.391667,48,1h,station08.csv,None


In [12]:
cl_results['evaluation']

,R^2,RMSE,MAE,Skill,output_dim,freq,key,t_0
Models,,,,,,,,
tft,0.754981,0.114394,0.059101,0.084248,48,1h,station00.csv,None
tft,0.856837,0.116063,0.066582,0.263583,48,1h,station01.csv,None
tft,0.771421,0.123712,0.070206,0.129800,48,1h,station02.csv,None
tft,0.892132,0.126291,0.067960,0.472879,48,1h,station04.csv,None
tft,0.396252,0.162357,0.091801,-0.589069,48,1h,station06.csv,None
tft,0.797792,0.110744,0.060261,0.209558,48,1h,station07.csv,None
tft,0.610182,0.159019,0.092399,-0.110543,48,1h,station08.csv,None
